In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [2]:
#ran = list(range(0,90))
#val = [np.sin(i) for i in ran]
#plt.plot(ran,val)

In [3]:
def generate_time_series(batch_size,n_steps):
    freq1, freq2, offsets1 , offsets2 = np.random.rand(4, batch_size,1)
    time = np.linspace(0,1,n_steps)
    series = 0.5* np.sin((time-offsets1) * (freq1*10 + 10)) #wave1
    series += 0.2* np.sin((time-offsets2) * (freq1*10 + 10)) #wave2
    series += 0.5* np.sin((time-offsets1) * (freq1*10 + 10)) #noise
    return series[...,np.newaxis].astype(np.float32) #ellipsis operator, used to select the entire series

Creating a training, test and calidation set

In [4]:
n_steps = 50
series = generate_time_series(10000,n_steps+1)
X_train, y_train = series[:7000,:n_steps],series[:7000,-1] 
X_valid , y_valid = series[7000:9000,:n_steps],series[7000:9000,-1]
X_test, y_test = series[9000:,:n_steps], series[9000:,-1]

We are considering the current output(y(t)) as the output feature while the past values are being used as inputs

In [5]:
np.shape(X_train)

(7000, 50, 1)

### Baselining metrics

Naive forecasting

In [6]:
y_pred = X_valid[:,-1]

np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.050076712

Using fully connected network

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[50,1]),
    keras.layers.Dense(1)
])

We are using flatten because it expects a flat list of features for each input

In [8]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
219/219 [==============================] - 2s 3ms/step - loss: 0.1699
Epoch 2/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0273
Epoch 3/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0185
Epoch 4/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0141
Epoch 5/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0103
Epoch 6/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0072
Epoch 7/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0049
Epoch 8/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0032
Epoch 9/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0020
Epoch 10/20
219/219 [==============================] - 1s 3ms/step - loss: 0.0013
Epoch 11/20
219/219 [==============================] - 1s 3ms/step - loss: 8.7627e-04
Epoch 12/20
219/219 [==============================] - 1s 4ms/step - loss: 6.4154e-04
Epoch 13/20
219/2

In [9]:
neural_pred = model.predict(X_valid)
np.mean(keras.losses.mean_squared_error(y_valid,neural_pred))

63/63 [==============================] - 0s 3ms/step


0.00034228928

### Implementing a Simple RNN

In [10]:
simpleRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape= [ None, 1])
])

We need not specify the length of the input Sequence since RNN can process any number of steps and hence the `None`

In [11]:
simpleRNN.compile(optimizer='adam',loss='mean_squared_error')
simpleRNN.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
219/219 [==============================] - 6s 14ms/step - loss: 0.5120
Epoch 2/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5102
Epoch 3/20
219/219 [==============================] - 4s 16ms/step - loss: 0.5117
Epoch 4/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5107
Epoch 5/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5096
Epoch 6/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5107
Epoch 7/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5090
Epoch 8/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5105
Epoch 9/20
219/219 [==============================] - 3s 16ms/step - loss: 0.5096
Epoch 10/20
219/219 [==============================] - 3s 13ms/step - loss: 0.5098
Epoch 11/20
219/219 [==============================] - 3s 14ms/step - loss: 0.5086
Epoch 12/20
219/219 [==============================] - 3s 13ms/step - loss: 0.5104
Epoch 13/20
2

In [12]:
simple_rnn_pred = simpleRNN.predict(X_valid)
np.mean(keras.losses.mean_squared_error(y_valid,simple_rnn_pred))

63/63 [==============================] - 1s 5ms/step


0.49795648

Implementing a deep RNN

In [13]:
deepRNN = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences= True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [14]:
deepRNN2 = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences= True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

In [16]:
deepRNN2.compile(optimizer='adam',loss='mean_squared_error')
deepRNN2.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
219/219 [==============================] - 9s 31ms/step - loss: 0.0266
Epoch 2/20
219/219 [==============================] - 6s 26ms/step - loss: 0.0014
Epoch 3/20
219/219 [==============================] - 5s 24ms/step - loss: 6.1379e-04
Epoch 4/20
219/219 [==============================] - 5s 24ms/step - loss: 3.4790e-04
Epoch 5/20
219/219 [==============================] - 5s 23ms/step - loss: 2.2381e-04
Epoch 6/20
219/219 [==============================] - 5s 24ms/step - loss: 1.6255e-04
Epoch 7/20
219/219 [==============================] - 5s 24ms/step - loss: 1.4055e-04
Epoch 8/20
219/219 [==============================] - 5s 23ms/step - loss: 1.0014e-04
Epoch 9/20
219/219 [==============================] - 5s 24ms/step - loss: 8.5285e-05
Epoch 10/20
219/219 [==============================] - 5s 24ms/step - loss: 7.9172e-05
Epoch 11/20
219/219 [==============================] - 5s 23ms/step - loss: 6.9131e-05
Epoch 12/20
219/219 [==============================] - 6s 26

In [15]:
deep_rnn_pred = deepRNN2.predict(X_valid)
np.mean(keras.losses.mean_squared_error(y_valid,deep_rnn_pred))

63/63 [==============================] - 1s 8ms/step


0.6733502

Forecasting several time steps ahead